# Create Agents to Research and Write an Article

In [31]:
#!pip install crewai==0.175.0 crewai_tools==0.65.0 langchain_community==0.3.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Import from the crewAI libray.
from crewai import Agent, Task, Crew, LLM

## Creating Agents

- Define Agents, and provide them a `role`, `goal` and `backstory`.
- Usually LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [33]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=LLM(model="ollama/llama3.2:3b", base_url="http://localhost:11434"),
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [34]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm=LLM(model="ollama/llama3.2:3b", base_url="http://localhost:11434"),
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [35]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=LLM(model="ollama/llama3.2:3b", base_url="http://localhost:11434"),
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [36]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [37]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [38]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this usecase*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=True` allows to see all the logs of the execution. 

In [39]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

In [40]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1760e9dc-2a91-429b-b64e-4d38341b1b1d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Comprehensive Content Plan Document for Blog Article on Artificial Intelligence                                │
│                                                                                                                 │
│  I. Introduction                                                                                                │
│   A. Brief overview of Artificial Intelligence (AI) and its increasing presence in modern technology            │
│   B. Importance of understanding AI and its impact on society                                                   │
│   C. Thesis statement: Artificial Intelligence is transforming industries, revolutionizing the way we live and  │
│  work, but its future holds both opportunities and challenges.                                                  │
│                                                                                                                 │
│  II. Latest Trends in Artificial Intelligence                                                                   │
│                                                                                                                 │
│   A. Explaination of Deep Learning:                                                                             │
│    1. Definition of deep learning                                                                               │
│    2. How it's being used in AI applications (e.g., image recognition, natural language processing)             │
│    3. Advantages and limitations of deep learning                                                               │
│                                                                                                                 │
│   B. Natural Language Processing (NLP):                                                                         │
│    1. Definition of NLP                                                                                         │
│    2. Applications of NLP in chatbots, sentiment analysis, and text summarization                               │
│    3. Challenges and future directions for NLP                                                                  │
│                                                                                                                 │
│   C. Edge AI:                                                                                                   │
│    1. Definition of edge AI                                                                                     │
│    2. Benefits of edge AI (e.g., real-time processing, reduced latency)                                         │
│    3. Use cases for edge AI in IoT devices, autonomous vehicles, and smart homes                                │
│                                                                                                                 │
│  III. Key Players in Artificial Intelligence                                                                    │
│                                                                                                                 │
│   A. Google DeepMind:                                                                                           │
│    1. Overview of DeepMind's AI research and applications                                                       │
│    2. Breakthroughs in areas like AlphaGo, AlphaFold, a

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 64e72932-376d-488d-ba8d-e877f088128f                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 70df58b9-2479-42ca-b006-fae2685e1064                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Introduction                                                                                                │
│  Artificial Intelligence (AI) has become an integral part of modern technology, transforming industries and     │
│  revolutionizing the way we live and work. As AI continues to advance at an unprecedented rate, it's essential  │
│  to understand its impact on society and its future implications. In this article, we'll delve into the latest  │
│  trends in AI, key players in the industry, and notable news that highlights the importance of AI development.  │
│                                                                                                                 │
│  ## Latest Trends in Artificial Intelligence                                                                    │
│                                                                                                                 │
│  ### Deep Learning                                                                                              │
│  Deep learning is a subset of machine learning that involves the use of neural networks with multiple layers    │
│  to analyze data. This technique has been widely adopted in AI applications such as image recognition, natural  │
│  language processing, and speech recognition. Deep learning offers several advantages over traditional machine  │
│  learning methods, including improved accuracy and scalability.                                                 │
│                                                                                                                 │
│  However, deep learning also has its limitations. For instance, it can be computationally expensive and         │
│  requires large amounts of data to train. Additionally, the lack of transparency in deep learning models makes  │
│  it challenging to understand how they arrive at their predictions.                                             │
│                                                                                                                 │
│  Despite these challenges, deep learning remains a critical component of AI development. Recent breakthroughs   │
│  in areas like AlphaGo and AlphaFold have pushed the boundaries of what is possible with deep learning.         │
│                                                                                                                 │
│  ### Natural Language Processing                                                                                │
│  Natural language processing (NLP) is the process of analyzing human language to extract insights and meaning.  │
│  NLP has become increasingly important in recent years, particularly with the rise of chatbots and virtual      │
│  assistants. NLP enables machines to understand and respond to human language, making it an essential           │
│  component of AI-powered applications.                                                                          │
│                                                                                                                 │
│  ### Edge AI                                                                                                    │
│  Edge AI refers to the processing of data at the edge of a network, rather than in a centralized location.      │
│  This approach offers several benefits, including reduc

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e745fdf0-f945-4f17-a335-1b87fa06b766                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1760e9dc-2a91-429b-b64e-4d38341b1b1d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Introduction                                                                                  │
│  Artificial Intelligence (AI) has become an integral part of modern technology, transforming industries and     │
│  revolutionizing the way we live and work. As AI continues to advance at an unprecedented rate, it's essential  │
│  to understand its impact on society and its future implications. In this article, we'll delve into the latest  │
│  trends in AI, key players in the industry, and notable news that highlights the importance of AI development.  │
│                                                                                                                 │
│  ## Latest Trends in Artificial Intelligence                                                                    │
│                                                                                                                 │
│  ### Deep Learning                                                                                              │
│  Deep learning is a subset of machine learning that involves the use of neural networks with multiple layers    │
│  to analyze data. This technique has been widely adopted in AI applications such as image recognition, natural  │
│  language processing, and speech recognition. Deep learning offers several advantages over traditional machine  │
│  learning methods, including improved accuracy and scalability.                                                 │
│                                                                                                                 │
│  However, deep learning also has its limitations. For instance, it can be computationally expensive and         │
│  requires large amounts of data to train. Additionally, the lack of transparency in deep learning models makes  │
│  it challenging to understand how they arrive at their predictions.                                             │
│                                                                                                                 │
│  Despite these challenges, deep learning remains a critical component of AI development. Recent breakthroughs   │
│  in areas like AlphaGo and AlphaFold have pushed the boundaries of what is possible with deep learning.         │
│                                                                                                                 │
│  ### Natural Language Processing                                                                                │
│  Natural language processing (NLP) is the process of analyzing human language to extract insights and meaning.  │
│  NLP has become increasingly important in recent years, particularly with the rise of chatbots and virtual      │
│  assistants. NLP enables machines to understand and respond to human language, making it an essential           │
│  component of AI-powered applications.                                                                          │
│                                                                                                                 │
│  ### Edge AI                                                                                                    │
│  Edge AI refers to the processing of data at the edge 

- Display the result as markdown

In [43]:
from IPython.display import Markdown, display
display(Markdown(str(result)))

## Introduction
Artificial Intelligence (AI) has become an integral part of modern technology, transforming industries and revolutionizing the way we live and work. As AI continues to advance at an unprecedented rate, it's essential to understand its impact on society and its future implications. In this article, we'll delve into the latest trends in AI, key players in the industry, and notable news that highlights the importance of AI development.

## Latest Trends in Artificial Intelligence

### Deep Learning
Deep learning is a subset of machine learning that involves the use of neural networks with multiple layers to analyze data. This technique has been widely adopted in AI applications such as image recognition, natural language processing, and speech recognition. Deep learning offers several advantages over traditional machine learning methods, including improved accuracy and scalability.

However, deep learning also has its limitations. For instance, it can be computationally expensive and requires large amounts of data to train. Additionally, the lack of transparency in deep learning models makes it challenging to understand how they arrive at their predictions.

Despite these challenges, deep learning remains a critical component of AI development. Recent breakthroughs in areas like AlphaGo and AlphaFold have pushed the boundaries of what is possible with deep learning.

### Natural Language Processing
Natural language processing (NLP) is the process of analyzing human language to extract insights and meaning. NLP has become increasingly important in recent years, particularly with the rise of chatbots and virtual assistants. NLP enables machines to understand and respond to human language, making it an essential component of AI-powered applications.

### Edge AI
Edge AI refers to the processing of data at the edge of a network, rather than in a centralized location. This approach offers several benefits, including reduced latency and improved security. Edge AI is particularly important for IoT devices, autonomous vehicles, and smart homes, where real-time processing is critical.

## Key Players in Artificial Intelligence

### Google DeepMind
Google DeepMind is one of the leading players in AI development, with a focus on machine learning and deep learning. The company's AlphaGo program has achieved several notable breakthroughs in areas like computer chess and Go.

DeepMind's AlphaFold project aims to predict the 3D structure of proteins, which could lead to new treatments for diseases. The company's research efforts have also focused on reinforcement learning and transfer learning.

### Microsoft Azure Machine Learning
Microsoft Azure Machine Learning (AML) is a cloud-based platform that enables developers to build and deploy machine learning models. AML offers several features, including automated machine learning and data science services.

AML has been adopted by several major companies, including Starbucks and Walmart, who have used the platform to improve their supply chain management and customer loyalty programs.

### NVIDIA
NVIDIA is a leading player in AI development, with a focus on GPU technology. The company's GPUs are specifically designed for AI workloads and have become an essential component of many AI applications.

NVIDIA has partnered with several major companies, including Google, Facebook, and Microsoft, to develop new AI technologies. The company's research efforts have also focused on areas like edge AI and autonomous vehicles.

## Noteworthy News in Artificial Intelligence

### Breakthroughs in Explainable AI
Explainable AI (XAI) refers to the process of understanding how machine learning models arrive at their predictions. XAI has become increasingly important as concerns about fairness, transparency, and trustworthiness grow.

Recent breakthroughs in XAI have led to new techniques like SHAP, LIME, and Anchors. These methods enable developers to understand how machine learning models work and identify potential biases.

### AI-Powered Healthcare
Artificial intelligence (AI) is being increasingly used in healthcare to improve diagnosis accuracy and patient outcomes. AI-powered medical imaging analysis has become a critical component of many hospitals' diagnostic processes.

Recent studies have shown that AI can be more accurate than human clinicians in certain areas, such as disease detection and treatment recommendations.

### Ethics and Governance of Artificial Intelligence
As AI becomes increasingly pervasive, concerns about ethics and governance are growing. The need for frameworks and regulations to ensure the responsible development and deployment of AI is becoming more pressing.

Recent initiatives and regulations, like the EU's AI Act and the US's Fairness, Accountability, and Transparency (FAT) initiative, aim to address these concerns and promote transparency and accountability in AI development.

## Target Audience Analysis

### Demographics
Our target audience includes individuals aged 25-45 years old with a college education and background in computer science or related fields. They are interested in staying up-to-date with the latest AI trends and technologies.

This audience is likely to be professionals working in industries like tech, finance, and healthcare. They have a strong interest in understanding how AI can improve their work and are willing to invest time and resources in learning about new technologies.

### Pain Points
Our target audience faces several pain points related to AI development, including:

* Concerns about job displacement and the need for continuous skill development
* Interest in staying up-to-date with the latest AI trends and technologies
* Desire to understand the social implications of AI on society

## Conclusion
Artificial intelligence is transforming industries and revolutionizing our lives. As we continue to advance at an unprecedented rate, it's essential to understand its impact on society and its future implications.

In this article, we've explored the latest trends in AI, key players in the industry, and notable news that highlights the importance of AI development. We've also analyzed our target audience and identified their pain points and interests.

As we look to the future, it's clear that AI will continue to play a critical role in shaping our world. By understanding its impact and implications, we can work towards creating a more equitable and transparent society for all.

## References
* MarketsandMarkets: "Artificial Intelligence Market by Technology (Deep Learning, Natural Language Processing, Computer Vision), Industry (Healthcare, Finance, Retail), Geography - Global Forecast to 2025"
* McKinsey: "A future that works: Automation, employment, and productivity"
* Gartner: "Market Trends Report: Artificial Intelligence, May 2019"